In [2]:
import random
import json
import pickle #for sterilisation
import numpy as np

import nltk
#lemmatizer is used in stemming words
from nltk.stem import WordNetLemmatizer 
nltk.download('punkt')
nltk.download('wordnet')


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.optimizers import SGD

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dagbo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dagbo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

words = []
classes = []
docs = []
ignore = [',',',','?','.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize is used in splitting words
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        docs.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore]
words = sorted(set(words))

print (docs)

classes = sorted(set(classes))


#save to file using pickle to dump
pickle.dump(words, open ('words.pkl', 'wb'))#wb means writing into binariries
pickle.dump(classes, open ('classes.pkl', 'wb'))

#represents the words as numerical values using bag_of_words
train = []
empty_output = [0] * len(classes)

for doc in docs:
    #create an empty list of bag of words
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns  else bag.append(0)
    
    output = list(empty_output)
    output[classes.index(doc[1])] = 1
    train.append([bag, output])
    
#shuffling train data   
random.shuffle(train)

train = np.array(train)

# create train and test lists. X - patterns, Y - intents
training_x = list(train[:, 0])
training_y = list(train[:, 1])

#building the neural network model
model = Sequential()
#adding layers to the model
model.add(Dense(128, input_shape=(len(training_x[0]),), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(training_y[0]), activation = 'softmax'))

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum = 0.9, nesterov = True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics = ['accuracy'])

bot = model.fit(np.array(training_x), np.array(training_y), epochs=200, batch_size = 5, verbose = 1)
model.save('Chatbot_model.h5', bot)
print("Finished...")
       


            


[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['i', 'feel', 'sick'], 'sick'), (['i', 'am', 'sick'], 'sick'), (['i', 'am', 'not', 'feeling', 'too', 'well'], 'sick'), (['i', 'am', 'not', 'well'],

<ipython-input-6-950ebbe4a4d4>:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train = np.array(train)


8/8 [==============================] - 0s 1ms/step - loss: 2.3775 - accuracy: 0.1000
Epoch 2/200
8/8 [==============================] - 0s 1ms/step - loss: 2.2009 - accuracy: 0.3000
Epoch 3/200
8/8 [==============================] - 0s 1ms/step - loss: 2.2745 - accuracy: 0.2250
Epoch 4/200
8/8 [==============================] - 0s 1ms/step - loss: 2.1568 - accuracy: 0.1750
Epoch 5/200
8/8 [==============================] - 0s 1ms/step - loss: 2.1342 - accuracy: 0.2750
Epoch 6/200
8/8 [==============================] - 0s 857us/step - loss: 2.0942 - accuracy: 0.2500
Epoch 7/200
8/8 [==============================] - 0s 1ms/step - loss: 2.0027 - accuracy: 0.3750
Epoch 8/200
8/8 [==============================] - 0s 857us/step - loss: 1.8568 - accuracy: 0.4750
Epoch 9/200
8/8 [==============================] - 0s 1ms/step - loss: 1.9003 - accuracy: 0.5000
Epoch 10/200
8/8 [==============================] - 0s 1ms/step - loss: 1.8309 - accuracy: 0.4250
Epoch 11/200
8/8 [===================

8/8 [==============================] - 0s 1ms/step - loss: 0.1117 - accuracy: 0.9750
Epoch 85/200
8/8 [==============================] - 0s 1000us/step - loss: 0.0616 - accuracy: 1.0000
Epoch 86/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0474 - accuracy: 1.0000
Epoch 87/200
8/8 [==============================] - 0s 1ms/step - loss: 0.1533 - accuracy: 0.9250
Epoch 88/200
8/8 [==============================] - 0s 1ms/step - loss: 0.1016 - accuracy: 0.9750
Epoch 89/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0824 - accuracy: 1.0000
Epoch 90/200
8/8 [==============================] - 0s 857us/step - loss: 0.0475 - accuracy: 1.0000
Epoch 91/200
8/8 [==============================] - 0s 1ms/step - loss: 0.1023 - accuracy: 0.9500
Epoch 92/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0841 - accuracy: 1.0000
Epoch 93/200
8/8 [==============================] - 0s 1000us/step - loss: 0.0454 - accuracy: 1.0000
Epoch 94/200
8/8 [=======

8/8 [==============================] - 0s 1ms/step - loss: 0.0537 - accuracy: 0.9750
Epoch 167/200
8/8 [==============================] - 0s 1ms/step - loss: 0.1318 - accuracy: 0.9750
Epoch 168/200
8/8 [==============================] - 0s 857us/step - loss: 0.0131 - accuracy: 1.0000
Epoch 169/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0340 - accuracy: 1.0000
Epoch 170/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0198 - accuracy: 1.0000
Epoch 171/200
8/8 [==============================] - 0s 857us/step - loss: 0.0235 - accuracy: 1.0000
Epoch 172/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0680 - accuracy: 0.9500
Epoch 173/200
8/8 [==============================] - 0s 857us/step - loss: 0.0136 - accuracy: 1.0000
Epoch 174/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0143 - accuracy: 1.0000
Epoch 175/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0236 - accuracy: 1.0000
Epoch 176/200
8/8 